In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from tqdm.notebook import tqdm

import sys
sys.path.append('../')
import default_style

TS_DATASET_FOLDER = os.path.join("..", "dataset")
TS_PREPROC_FOLDER = os.path.join(TS_DATASET_FOLDER, "preprocessed_traces")
DF_PREPROC_FILE = os.path.join(TS_PREPROC_FOLDER, "preproc_ts.df")

INTERESTING_TRACES = ["denoised_trace", "std_phon_trace", "phonetic_trace"]

## Save folder for long-run estimators
ESTIMATORS_FOLDER = os.path.join("..", "estimators")

In [ ]:
# Load stuff
df = pd.read_csv(DF_PREPROC_FILE)

traces = dict()
for t in INTERESTING_TRACES:
    traces[t] = np.load(os.path.join(TS_PREPROC_FOLDER, f"{t}.npy"))

In [ ]:
# Minimal preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

X = traces["denoised_trace"]
X[X==np.nan] = 0

ohenc = OneHotEncoder()
y = ehenc.fit_transform(df.emotion)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Define the model
from keras.layers import LSTM, GRU, Conv1D, Input, Dense
from keras.metrics import CategoricalCrossentropy as cat_cross_metric, CategoricalAccuracy as cat_acc_metric
from keras.losses import CategoricalCrossentropy
from keras.models import model

# Model
input_layer = Input(shape=X.shape[1], name="input")
lstm = LSTM(128)(input_layer)
dense = Dense(128)(lstm)
dense = Dense(128)(dense)
output = Dense(y.shape[1], activation="softmax", name="output")(dense)

# Compilation
model = Model(inputs=input, outputs=output, name="LSTM_dense")
model.compile(optimizer="adam",
            loss=CategoricalCrossentropy(),
            metrics= [cat_cross_metric(), cat_acc_metric()],

model.summary()

In [ ]:
# Fit
history = model.fit(
                    X_train, y_train, 
                    epochs=10, batch_size=10, verbose=1, 
                    validation_split = 0.05
                   )